<a href="https://colab.research.google.com/github/e-olang/NLP/blob/main/Swahili/Distillation/draft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install accelerate -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 17.0 MB/s eta 0:00:00


In [3]:
!pip install datasets transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 22.5 MB/s eta 0:00:00


In [4]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, DistilBertForMaskedLM, Trainer, TrainingArguments

In [5]:
# Load pre-trained model and tokenizer
teacher_model = AutoModelForMaskedLM.from_pretrained("eolang/SW-v1")
tokenizer = AutoTokenizer.from_pretrained("eolang/SW-v1")

In [6]:
# Load dataset
from datasets import load_dataset
dataset = load_dataset("swahili")
dataset

Generating train split:   0%|          | 0/42069 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3371 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3372 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 42069
    })
    test: Dataset({
        features: ['text'],
        num_rows: 3371
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3372
    })
})

In [7]:
print(dataset['train']['text'][0])

taarifa hiyo ilisema kuwa ongezeko la joto la maji juu ya wastani katikati ya bahari ya UNK inaashiria kuwepo kwa mvua za el nino UNK hadi mwishoni mwa april ishirini moja sifuri imeelezwa kuwa ongezeko la joto magharibi mwa bahari ya hindi linatarajiwa kuhamia katikati ya bahari hiyo hali ambayo itasababisha pepo kutoka kaskazini mashariki kuvuma kuelekea bahari ya hindi


In [8]:
# Tokenize dataset
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [9]:
tokenized_dataset = dataset.map(tokenize_function, batched=True, num_proc=4)

Map (num_proc=4):   0%|          | 0/42069 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/3371 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/3372 [00:00<?, ? examples/s]

In [10]:
# Train student model
student_model = DistilBertForMaskedLM.from_pretrained("distilbert-base-uncased")
training_args = TrainingArguments(
    output_dir="./distilbert",
    evaluation_strategy="steps",
    eval_steps=500,
    save_steps=500,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=1,
)

trainer = Trainer(
    model=student_model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
)

In [11]:
trainer.train()

ValueError: ignored